In [1]:
import pandas as pd 
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt
import re
from nltk.corpus import stopwords

test = pd.read_csv("test (1).csv")
train = pd.read_csv("train (1).csv")

In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5346 entries, 0 to 5345
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5346 non-null   int64 
 1   keyword   5311 non-null   object
 2   location  3590 non-null   object
 3   text      5346 non-null   object
 4   target    5346 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 209.0+ KB


In [33]:
train_2 = train.drop(["id","location","keyword"],axis=1)
test_2 = test.drop(["location","keyword"],axis=1)
train_2.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,#RockyFire Update => California Hwy. 20 closed...,1
4,I'm on top of the hill and I can see a fire in...,1


In [4]:
signos = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)|(\=)|(\>)|(\<)|(#)")

def signs_tweets(tweet):
    return signos.sub('', tweet.lower())

train_2['text'] = train_2['text'].apply(signs_tweets)
test_2['text'] = test_2['text'].apply(signs_tweets)

def remove_links(df):
    return " ".join(['{link}' if ('http') in word else word for word in df.split()])

train_2['text'] = train_2['text'].apply(remove_links)
test_2['text'] = test_2['text'].apply(remove_links)
train_2['text'].head()

0    our deeds are the reason of this earthquake ma...
1                forest fire near la ronge sask canada
2    all residents asked to 'shelter in place' are ...
3    rockyfire update california hwy closed in both...
4    i'm on top of the hill and i can see a fire in...
Name: text, dtype: object

In [5]:
spanish_stopwords = stopwords.words("english")

def remove_stopwords(train_2):
    return " ".join([word for word in train_2.split() if word not in spanish_stopwords])

train_2["text"] = train_2["text"].apply(remove_stopwords)
test_2["text"] = test_2["text"].apply(remove_stopwords)
train_2.head()

,text,target
0,deeds reason earthquake may allah forgive us,1
1,forest fire near la ronge sask canada,1
2,residents asked 'shelter place' notified offic...,1
3,rockyfire update california hwy closed directi...,1
4,i'm top hill see fire woods,1


In [6]:
train_2.head(23)

,text,target
0,deeds reason earthquake may allah forgive us,1
1,forest fire near la ronge sask canada,1
2,residents asked 'shelter place' notified offic...,1
3,rockyfire update california hwy closed directi...,1
4,i'm top hill see fire woods,1
5,three people died heat wave far,1
6,flood bago myanmar arrived bago,1
7,damage school bus multi car crash breaking,1
8,what's man,0
9,love fruits,0


In [7]:
def clean_text(X):
    a = X.split()
    X_new = [x for x in a if not x.startswith("@")]
    X_new = [x for x in a if not x.startswith("{link}")]
    frase = ' '.join(X_new)
    return frase

train_2['text'] = train_2['text'].apply(clean_text)
test_2['text'] = test_2['text'].apply(clean_text)

In [8]:
train_2.head(23)

,text,target
0,deeds reason earthquake may allah forgive us,1
1,forest fire near la ronge sask canada,1
2,residents asked 'shelter place' notified offic...,1
3,rockyfire update california hwy closed directi...,1
4,i'm top hill see fire woods,1
5,three people died heat wave far,1
6,flood bago myanmar arrived bago,1
7,damage school bus multi car crash breaking,1
8,what's man,0
9,love fruits,0


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import xml.etree.ElementTree as ET
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [22]:
vectorizer = CountVectorizer()
rand_forest = LogisticRegression()

pipeline = Pipeline([('vect', vectorizer), ('reglog', rand_forest)])


# Aqui definimos el espacio de parámetros a explorar
parameters = {
    'vect__max_df': ( 1.0, 0.8),
    'vect__min_df': (15, 12),
    'vect__max_features': (100,300,400),
    'vect__ngram_range': ((1,1),(1,2)),  # unigramas or bigramas
     "reglog__penalty": ["l1","l2"],
     "reglog__C": np.logspace(0, 4, 10)
}

grid_search = GridSearchCV(pipeline,
                           parameters,
                           cv=5,
                           n_jobs=-1,
                           scoring= 'roc_auc')


In [23]:
grid_search.fit(train_2['text'], train_2['target'])

/Users/cmg/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.68096266 0.68067762 0.68132181 0.68298625 0.72275674 0.72162999
 0.72260977 0.7230053  0.72662622 0.72537901 0.72512801 0.72447387
 0.68096266 0.68067762 0.68132181 0.68298625 0.72275674 0.72162999
 0.72260977 0.7230053  0.72662622 0.72537901 0.72512801 0.72447387
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.67926324 0.67851203 0.67952492 0.6827309

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('reglog', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'reglog__C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                         'reglog__penalty': ['l1', 'l2'],
                         'vect__max_df': (1.0, 0.8),
                         'vect__max_features': (100, 300, 400),
                         'vect__min_df': (15, 12),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             scoring='roc_auc')

In [28]:
best_grids = [(i, j.best_score_) for i, j in grids.items()]
best_grids = pd.DataFrame(best_grids,
                          columns=['Grid',
                                   'Best score']).sort_values(by='Best score',
                                                              ascending=False)

In [29]:
grid_search.best_score_

0.7266262198292411

In [30]:
predictions_proba = grid_search.predict_proba(test_2['text'])

In [32]:
test_2

,text
0,people receive wildfires evacuation orders cal...
1,got sent photo ruby alaska smoke wildfires pou...
2,flood disaster heavy rain causes flash floodin...
3,there's emergency evacuation happening buildin...
4,i'm afraid tornado coming area
...,...
2262,mh aircraft debris found la reunion missing ma...
2263,evacuation order lifted town roosevelt
2264,breaking la refugio oil spill may costlier big...
2265,stormchase violent record breaking ef- el reno...


In [34]:
submission = test_2[['id']]
submission['target']= predictions_proba[:,1]

submission.to_csv('submission.csv', index =False)